## Match searches with bookings

- For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match. 

- For the bookings file, origin and destination are the columns dep_port and arr_port, respectively. 

- Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.



## 2) Prepare the data for processing

### Booking

#### We didnt check for duplicates so far... What if the file is has duplicated lines?

In [2]:
ls

bookings.csv.bz2         ch_03-Empty.ipynb   searches.csv.bz2
bookings.sample.csv.bz2  ch_04-Empty.ipynb   top_airports.csv
ch_01-Empty.ipynb        ch_05b-empty.ipynb  Untitled.ipynb
ch_02-Empty.ipynb        ch_05-empty.ipynb


In [1]:
import pandas as pd

In [2]:
!pwd

/Users/aalmagro/Documents/KSchool/023/Amadeus_challenge


In [5]:
path_bookings = 'challenge/bookings.csv.bz2'
path_searches = 'challenge/searches.csv.bz2'
path_bookings_no_dup = 'bookings_wn_dup.csv'
path_searches_no_dup ='searches_wn_dup.csv'

In [3]:
%%time
bi=pd.read_csv(path_bookings, sep="^", 
               chunksize=100000, low_memory=False, 
               dtype=str)
all_chunks=pd.DataFrame()

for i, chunk in enumerate(bi):
    print(i, len(all_chunks))
    all_chunks=all_chunks.append(chunk)
    all_chunks.drop_duplicates(inplace=True)
all_chunks.columns=all_chunks.columns.str.strip()   
all_chunks.to_csv(path_bookings_no_dup, sep='^', 
                  index=False)

0 0
1 100000
2 200000
3 300000
4 400000
5 500000
6 600000
7 700000
8 800000
9 900000
10 1000000
11 1000000
12 1000000
13 1000000
14 1000000
15 1000000
16 1000000
17 1000000
18 1000000
19 1000000
20 1000000
21 1000000
22 1000000
23 1000000
24 1000000
25 1000000
26 1000000
27 1000000
28 1000000
29 1000000
30 1000000
31 1000000
32 1000000
33 1000000
34 1000000
35 1000000
36 1000000
37 1000000
38 1000000
39 1000000
40 1000000
41 1000000
42 1000000
43 1000000
44 1000000
45 1000000
46 1000000
47 1000000
48 1000000
49 1000000
50 1000000
51 1000003
52 1000003
53 1000003
54 1000003
55 1000003
56 1000003
57 1000003
58 1000003
59 1000003
60 1000003
61 1000003
62 1000003
63 1000003
64 1000003
65 1000003
66 1000003
67 1000003
68 1000003
69 1000003
70 1000003
71 1000003
72 1000003
73 1000003
74 1000003
75 1000003
76 1000003
77 1000003
78 1000003
79 1000003
80 1000003
81 1000003
82 1000003
83 1000003
84 1000003
85 1000003
86 1000003
87 1000003
88 1000003
89 1000003
90 1000003
91 1000003
92 1000003
93

In [9]:
%%time
bi=pd.read_csv(path_searches, sep="^", chunksize=500000, low_memory=False, dtype=str)
all_chunks=pd.DataFrame()

for i, chunk in enumerate(bi):
    print(i, len(all_chunks))
    all_chunks=all_chunks.append(chunk)
    all_chunks.drop_duplicates(inplace=True)
    if i==2:
        break
all_chunks.columns=all_chunks.columns.str.strip()
all_chunks["Origin"]=all_chunks["Origin"].str.strip().str.upper()
all_chunks["Destination"]=all_chunks["Destination"].str.strip().str.upper()
    
all_chunks.to_csv(path_searches_no_dup, sep='^', index=False)

0 0
1 358999
2 358999
CPU times: user 25.3 s, sys: 1.61 s, total: 26.9 s
Wall time: 27.6 s


In [10]:
Bookings Columns:'act_date           ', 'dep_port', 'arr_port'
Search Columns:'Date', 'Origin', 'Destination'

SyntaxError: invalid syntax (<ipython-input-10-97d6de3516fd>, line 1)

#### We have seen that we have white space in some columns....

In [21]:
cols_bookings=['cre_date','act_date','dep_port', 'arr_port']
b=pd.read_csv(path_bookings_no_dup, sep='^', usecols=cols_bookings)

In [22]:
s=pd.read_csv(path_searches_no_dup, sep='^', low_memory=False)

In [23]:
b.columns

Index(['act_date', 'cre_date', 'dep_port', 'arr_port'], dtype='object')

In [24]:
b.arr_port=b.arr_port.str.strip()
b.dep_port=b.dep_port.str.strip()

In [25]:
b.arr_port[0]

'LHR'

In [26]:
b.act_date[0]

'2013-03-05 00:00:00'

In [ ]:
Action plan:
1) quitar duplicates
2) prepare act_date in the date format of the search
3) add columns with 1s in bookings
4) merge --> left join
Bookings Columns: 'act_date', 'dep_port', 'arr_port'
Search Columns:'Date', 'Origin', 'Destination'
5)fillna(0) over the result
6) save

In [27]:
import pandas as pd
b=pd.read_csv(path_bookings_no_dup, sep='^', usecols=cols_bookings)
s=pd.read_csv(path_searches_no_dup, sep='^', low_memory=False)

In [28]:
b.drop_duplicates(inplace=True)
b.columns=b.columns.str.strip()
b.arr_port=b.arr_port.str.strip()
b.dep_port=b.dep_port.str.strip()
b.cre_date=b.cre_date.str[0:10]
s.Date=s.Date.str[0:10]
b['Booked']=1

s_b=s.merge(b,
            how='left', 
            left_on=['Date', 'Origin', 'Destination'],
            right_on=['cre_date', 'dep_port', 'arr_port'])



In [29]:
b.head()

,act_date,cre_date,dep_port,arr_port,Booked
0,2013-03-05 00:00:00,2013-02-22,ZRH,LHR,1
1,2013-03-26 00:00:00,2013-03-26,SAL,CLT,1
3,2013-03-26 00:00:00,2013-03-26,AKL,SVO,1
5,2013-03-20 00:00:00,2013-03-20,DEN,LGA,1
7,2013-03-25 00:00:00,2013-03-25,NRT,SIN,1


In [30]:
len(s), len(s_b)

(359003, 366204)

In [57]:
s_b.drop(['act_date', 'dep_port', 'arr_port'], axis=1, inplace=True)

In [67]:
s_b['Booked']=s_b['Booked'].fillna(0)

In [68]:
s_b=s_b.astype({'Booked':int})

In [70]:
s_b[ s_b['Booked']==1 ].sample(5)

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,cre_date,dep_port,arr_port,Booked
167636,2013-06-17,11:34:42,FXX,0a914cfb66b849dddaa6bb2e66ff0999,DE,TXL,BKK,1.0,4.0,TXL,...,1ASI,0,0,0,a2f6efe42c20a759e30deafe68890442,BNJ,2013-06-17,TXL,BKK,1
301236,2013-10-29,14:49:11,MPT,0c618d30c252b880d8a83ebb76af7643,SE,LIM,CUZ,1.0,2.0,LIM,...,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,0,2013-10-29,LIM,CUZ,1
161484,2013-06-11,00:57:26,MPT,19d81e19f1342e305d888f236565d20d,ES,MAD,LPA,0.0,1.0,MAD,...,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,MAD,2013-06-11,MAD,LPA,1
195553,2013-07-15,06:48:32,MPT,966af57bac7a9b07b029ebb01e0e9574,GR,BRU,BKK,1.0,2.0,BRU,...,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,ATH,2013-07-15,BRU,BKK,1
139092,2013-05-20,02:34:34,FCP,1722290df1e162d326280c5d4a82657b,CO,BOG,CTG,0.0,1.0,BOG,...,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,BOG,2013-05-20,BOG,CTG,1


In [71]:
import pandas as pd
b=pd.read_csv('bookings_wn_dup.csv', sep='^', usecols=['act_date           ', 'dep_port', 'arr_port'])
s=pd.read_csv('searches_wn_dup.csv', sep='^', low_memory=False)
b.drop_duplicates(inplace=True)
b.columns=b.columns.str.strip()
b.arr_port=b.arr_port.str.strip()
b.dep_port=b.dep_port.str.strip()
b.act_date=b.act_date.str[0:10]
s.Date=s.Date.str[0:10]
b['Booked']=1

s_b=s.merge(b,
            how='left', 
            left_on=['Date', 'Origin', 'Destination'],
            right_on=['act_date', 'dep_port', 'arr_port'])

s_b['Booked']=s_b['Booked'].fillna(0)
s_b.drop(['act_date', 'dep_port', 'arr_port'], axis=1, inplace=True)
s_b=s_b.astype({'Booked':int})
s_b[ s_b['Booked']==1 ].sample(5)

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,act_date,dep_port,arr_port,Booked
310709,2013-11-07,16:47:19,FFP,1b4eb15d0f0f667cbc825602d09fff65,AU,MEL,LHR,1.0,2.0,MEL,...,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,0,2013-11-07,MEL,LHR,1
169915,2013-06-19,22:50:36,MPT,3561a60621de06ab1badc8ca55699ef3,US,ATL,MIA,1.0,2.0,ATL,...,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC,2013-06-19,ATL,MIA,1
10334,2013-01-11,10:48:28,MPT,788e52ddaa551032c69ba37cc316233f,FR,FRA,BKK,1.0,2.0,FRA,...,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,PAR,2013-01-11,FRA,BKK,1
137768,2013-05-18,02:17:38,MPT,8b4fa4e121637f583bfa86ef0fa26c9e,US,LAS,EWR,1.0,2.0,LAS,...,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,LAS,2013-05-18,LAS,EWR,1
21346,2013-01-22,13:11:27,MTP,236cbf458f79dbb9d5e9c9430438c3db,US,SFO,PVG,1.0,2.0,SFO,...,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,HPN,2013-01-22,SFO,PVG,1


In [72]:
s_b.to_csv('searches_with_bookings.csv', sep='^', index=False)

In [73]:
! head ./searches_with_bookings.csv

Date^Time^TxnCode^OfficeID^Country^Origin^Destination^RoundTrip^NbSegments^Seg1Departure^Seg1Arrival^Seg1Date^Seg1Carrier^Seg1BookingCode^Seg2Departure^Seg2Arrival^Seg2Date^Seg2Carrier^Seg2BookingCode^Seg3Departure^Seg3Arrival^Seg3Date^Seg3Carrier^Seg3BookingCode^Seg4Departure^Seg4Arrival^Seg4Date^Seg4Carrier^Seg4BookingCode^Seg5Departure^Seg5Arrival^Seg5Date^Seg5Carrier^Seg5BookingCode^Seg6Departure^Seg6Arrival^Seg6Date^Seg6Carrier^Seg6BookingCode^From^IsPublishedForNeg^IsFromInternet^IsFromVista^TerminalID^InternetOffice^act_date^dep_port^arr_port^Booked
2013-01-01^20:25:57^MPT^624d8c3ac0b3a7ca03e3c167e0f48327^DE^TXL^AUH^1.0^2.0^TXL^AUH^2013-01-26^D2^^AUH^TXL^2013-02-02^D2^^^^^^^^^^^^^^^^^^^^^^1ASIWS^0^0^0^d41d8cd98f00b204e9800998ecf8427e^FRA^^^^0
2013-01-01^10:15:33^MPT^b0af35b31588dc4ab06d5cf2986e8e02^MD^ATH^MIL^0.0^1.0^ATH^MIL^2013-01-04^^^^^^^^^^^^^^^^^^^^^^^^^^^^1ASIWS^0^0^0^d41d8cd98f00b204e9800998ecf8427e^KIV^^^^0
2013-01-01^18:04:49^MPT^3561a60621de06ab1badc8ca55699ef3^US^ICT